In [ ]:
#-----------------------------------------------------------------------------------
# This script solves (1) quasi-static equation of solid equilibrium "Navier's Eq."
# and (2) pore fluid pressure diffusion equation "Mass balance" with the
# Finite Element Method through Fenicsx open-source simulator. This simulation 
# is three-dimensional and employs the implicit Euler method of time-stepping. 
# The solution scheme is fully-coupled, requiring Newton iteration on the 
# dependent variables at each time step. Compressive stress is positive.
#-----------------------------------------------------------------------------------

#-----------------------------------------------------------------------------------
# Workflow: Gmsh meshing software --> Fenicsx simulator --> Paraview visualization
#-----------------------------------------------------------------------------------

#-----------------------------------------------------------------------------------
# Reservoir injection/depletion poroelastic response. Change Dirichlet/Neumann BCs
# for pressure diffusion equation to allow for reservoir boundaries, e.g., the 
# "East" boundary could be no flux to simulate a sealing fault.
#-----------------------------------------------------------------------------------

The solution method employs mixed finite-element scheme with dependent variables: displacement $\vec{u}=\left[u,v,w\right]$, Darcy flux $\vec{q}=\left[q_x,q_y,q_z\right]$, and pore pressure $p$. We use continuous linear piecewise elements for displacements, lowest-order Brezzi-Douglas-Martin elements for Darcy flux, and discontinuous piecewise constant elements for pore pressure. The complete formulation can be found elsewhere (Ref. [1]).

[1] Ferronato, M., Castelletto, N., & Gambolati, G. (2010). A fully coupled 3-D mixed finite element model of Biot consolidation. Journal of Computational Physics, 229(12), 4813-4830.

![WorkFlow](Workflow.jpeg)
Source: https://computationalmechanics.in/fenics-the-mesh-workflow/

In [ ]:
# Mesh from Gmsh (if mesh properties are updated, then .csv file must also be updated)
#-----------------------------------------------------------------------------------
from mesh_creation import Gmsh_model # Import gmsh model to Fenicsx
xl, yl, zl, ms, msw = 1e3, 1e3, 1e2, 5e1, 10e0 # x-dim, y-dim, z-dim, res. mesh size, well mesh size
model,gdim = Gmsh_model(xl, yl, zl, ms, msw) # Get the gmsh.model and dimension (2D or 3D)
from mesh_conversion import msh_to_xdmf # Import mesh conversion
mesh = msh_to_xdmf(model, gdim) # Fenicsx converted mesh
import pandas as pd # .csv file reader
import numpy as np # linear algebra tool --> similar to .* and ./ in matlab for matrix multiplication
df = pd.read_csv(r'Well_local_nodes.csv') # Injector/Producer well dofs (get this from Paraview)
Cx, Cy, Cz = np.array([df.Cx]), np.array([df.Cy]), np.array([df.Cz]) # well cell centers

In [ ]:
# Import several Fenicsx/dolfinx packages
#-----------------------------------------------------------------------------------
from dolfinx import plot, io
from dolfinx.fem import (dirichletbc, Expression, Function, FunctionSpace, 
                         VectorFunctionSpace, TensorFunctionSpace, locate_dofs_topological, Constant, locate_dofs_geometrical)
from dolfinx.fem.petsc import LinearProblem, NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.mesh import locate_entities_boundary, locate_entities, meshtags
from ufl import (TestFunction, TrialFunction, dot, dx, grad, inner, nabla_div, div, Identity, sym, Measure,
                 SpatialCoordinate, lhs, rhs, as_vector, tr, FiniteElement, VectorElement, MixedElement, 
                 split, FacetNormal, TensorElement, as_matrix, as_tensor, atan_2, cos, sin, nabla_grad, conditional, And, eq)
from petsc4py.PETSc import ScalarType, Options
import dolfinx
from mpi4py import MPI

In [ ]:
# Time stepping properties (Implicit Euler method)
#-----------------------------------------------------------------------------------
t = 0.0 # initialize time [s]
tmax = 86400*200 # 1 day = 86,400 seconds (units of time are seconds!)
num_steps = 50 # number of steps
dt = tmax/num_steps # time increment

# Reservoir properties
#-----------------------------------------------------------------------------------
depth = 2000 # Reservoir depth [m]
Temp = 15 + 0.025*depth # Reservoir temperature [degC]
g_cst = 9.81 # Gravity constant [m/s/s]
eps_H = 5e-4 # Tectonic strain for maximum horizontal stress
eps_h = 2.5e-4 # Tectonic strain for minimum horizontal stress

# Defined poroelastic constants
#-----------------------------------------------------------------------------------
E = 10e9 # Young's modulus [Pa]
nu = 0.22 # Poisson's ratio [-]
phi = 0.2 # Porosity [-]
k_ = 1e-13 # Permeability [m^2]
mu_ = 2.414e-5*10**(247.8/(Temp+273.15-140)) # Temperature dependent viscosity [Pa*s]
rho_f = 999.83311 + 0.0752*Temp - 0.0089*Temp**2 + 7.36413e-5*Temp**3 - 4.74639e-7*Temp**4 + 1.34888e-9*Temp**5 # Density [kg/m^3]
Ks = 37e9 # Solid bulk modulus [Pa]
Kf = 2.15e9 # Fluid bulk modulus [Pa]

# Derived poroelastic constants
#-----------------------------------------------------------------------------------
lambda_ = E*nu/((1+nu)*(1-2*nu)) # Lame parameter [Pa]
mu = E/(2*(1+nu)) # Shear modulus [Pa]
kappa = k_/mu_ # Bulk fluid mobility [m^2/Pa/s]
Km = E/(3*(1-2*nu)) # Drained bulk modulus [Pa]
alpha = 1-Km/Ks # Biot coefficient [-]
M = (phi/Kf + (alpha-phi)/Ks)**(-1) # Biot modulus [Pa]
Ku = Km + alpha**2 * M # Undrained bulk modulus [Pa]
nu_u = (3*Ku-2*mu)/(2*(3*Ku+mu)) # Undrained Poisson's ratio [-]

In [ ]:
# FEM spacev--> Mixed FEM Scheme for Poroelasticity
#-----------------------------------------------------------------------------------
disp = VectorElement("CG", mesh.ufl_cell(), 1) # Vector piecewise linear Lagrange element for u=(u,v,w)
pres = FiniteElement("DG", mesh.ufl_cell(), 0) # Scalar piecewise linear element for p
flux = FiniteElement("BDM", mesh.ufl_cell(), 1) # Vector Brezzi-Douglas-Martin element for q
ten = TensorElement("DG", mesh.ufl_cell(), 0) # Tensor piecewise linear element for S=(Sxx, Sxy, Sxz,
#                                                                                      Syx, Syy, Syx,
#                                                                                      Szx, Szy, Szz)
V0 = MixedElement([disp, pres, flux]) # Solution vector is ordered sol=(u,v,w,p,q)
V0 = FunctionSpace(mesh,V0) # Mixed function space
S = FunctionSpace(mesh, ten) # Tensor function space
C = FunctionSpace(mesh, pres) # Scalar function space

In [ ]:
# Initial vertical stress and fluid pressure --> horizontal stresses calculated
# by boundary conditions, e.g., tectonic strains and zero displacements.
#-----------------------------------------------------------------------------------
Sv = 22.62e3*depth # Total vertical stress [Pa]
Pp = 9.81e3*depth # Pore pressure [Pa]

In [ ]:
# Stress and strain definition
#-----------------------------------------------------------------------------------
def epsilon(u):
    e = sym(grad(u)) # epsilon = 0.5*(grad(u) + grad(u).T)
    return e
def sigma(u,p):
    sig = -lambda_*nabla_div(u)*Identity(u.geometric_dimension()) - 2*mu*sym(grad(u)) + alpha*(p)*Identity(u.geometric_dimension()) # Total stress tensor
    return sig
def sigma_eff(u,p):
    sig = -lambda_*nabla_div(u)*Identity(u.geometric_dimension()) - 2*mu*sym(grad(u)) + alpha*(p)*Identity(u.geometric_dimension()) - p*Identity(u.geometric_dimension()) # Effective stress tensor
    return sig
def epsilon_v(u):
    return nabla_div(u) # Volumetric strain
def porosity(u,p):
    delta_phi = phi + alpha*nabla_div(u) + (p-Pp)/M
    return delta_phi

In [ ]:
# Split mixed FEM space
#-----------------------------------------------------------------------------------
V = TestFunction(V0) # Test function of mixed element space
del_u, del_p, del_q = split(V) # Test variables for disp. and pressure

#-----------------------------------------------------------------------------------
# We now need a way to keep track of current and previous solutions
# to discretize time derivatives in fluid mass balance
#-----------------------------------------------------------------------------------

wk1 = Function(V0) # Current solution vector
wk = Function(V0) # Previous solutiun vector

uk1, pk1, qk1 = split(wk1) # Split into seperate variables
uk, pk, qk = split(wk) # Split into seperate variables

# Initial conditions for displacement, pressure, and flux
#-----------------------------------------------------------------------------------
wk1.x.array[:] = 0.0 # Initialize 1st time step
wk1.sub(1).interpolate(lambda x: Pp*np.ones(x.shape[1])) # Access to pressure IC
wk1.x.scatter_forward()

$$\begin{equation}\int_{\Omega}\left[\rho_f\alpha\left(\varepsilon_v^{k+1}-\varepsilon_v^{k}\right)\cdot\delta p\right] \mathrm{d}\Omega + \int_{\Omega}\left[\frac{\alpha^2}{K_u-K_d}\rho_f\left(p^{k+1}-p^k\right)\cdot\delta p\right]\mathrm{d}\Omega + \int_{\Omega}\left[\Delta t \rho_f \nabla \cdot \left(q^{k+1}\right) \delta p\right]\mathrm{d}\Omega = \int_{\Omega}\left[\Delta t\cdot f\left(\vec{x}-\vec{x}_w\right)\delta p\right]\mathrm{d}\Omega \tag{1}\end{equation}$$

$$\begin{equation}\int_{\Omega}\left[\frac{\mu}{k}q^{k+1}\cdot \delta q\right]\mathrm{d}\Omega = \int_{\Omega}\left[p^{k+1}\nabla \cdot\left(\delta q\right)\right]\mathrm{d}\Omega - \int_{\partial\Omega}\left[p^{k+1}_{BC}\left(\delta q \cdot \vec{n}\right)\right]\mathrm{d}\partial\Omega \tag{2}\end{equation}$$

$$\begin{equation}\int_{\Omega}\left[\sigma : \nabla \delta \vec{u}\right]\mathrm{d}\Omega = \int_{\partial\Omega}\left[\sigma_{BC}\cdot \vec{n}\right]\mathrm{d}\partial\Omega \tag{3}\end{equation}$$

where $\left(\vec{x}-\vec{x}_w\right)$ is the dirac delta function to restrict the degrees of freedom to the wellbore nodes. Hence, the value of $f$ is zero everywhere in the domain $\Omega$ except for the nodes aligning with the wellbore. The value of $f$ at the wellbore nodes is equal to the mass flow rate in kg/s.$\newline$

Boundary Condition Implementation: $\newline$
(1) Dirichlet pressure on $\partial\Omega$ --> weak-form implementation through Eq. 2 $\newline$
(2) No-flux on $\partial\Omega$ --> dirichlet implementation through Eq. 2 $\newline$
(3) Internal mass source $f$ --> weak-form dirac delta implementation through Eq. 1 $\newline$
(4) External loads on $\partial\Omega$ --> weak-form implementation through Eq. 3

In [ ]:
# Solve linear variational problem
#-----------------------------------------------------------------------------------
a11 = rho_f*alpha*epsilon_v(uk1)*del_p*dx + alpha**2/(Ku-Km)*rho_f*pk1*del_p*dx + dt*rho_f*div(qk1)*del_p*dx # Pressure diffusion eq
b11 = -rho_f*alpha*epsilon_v(uk)*del_p*dx - alpha**2/(Ku-Km)*rho_f*pk*del_p*dx # Pressure diffusion eq
c11 = (1/kappa)*inner(qk1,del_q)*dx - pk1*div(del_q)*dx # Flux continuity eq
d11 = inner(sigma(uk1,pk1),epsilon(del_u))*dx # Navier's eq
F = a11+b11+c11+d11 # Equation residual

In [ ]:
# Boundary conditions
#-----------------------------------------------------------------------------------
x = SpatialCoordinate(mesh) # Locate x,y,z coordinates of elements
n = -FacetNormal(mesh) # normal direction to elements
norm = as_vector([n[0], n[1], n[2]]) # This is needed for Neumann BCs

flow_rate = -100/Cx.shape[1] # Total flow rate [bbl/day] **Negative for depletion, positive for injection
flow_rate = flow_rate*1.84e-6 # Convert [bbl/day] to [m^3/s]
flow_rate = flow_rate*rho_f # Mass source [m^3/s] --> [kg/s]

#-----------------------------------------------------------------------------------
# Note: The mass source must be added to the weak form equation through Dirac Delta
# Function, e.g., f = m*delta(x-x_w). Therefore, we restrict the DOFs to only those 
# that align with "x_w" as defined in the .csv file.
#-----------------------------------------------------------------------------------

ff_space, _ = V0.sub(1).collapse() # Collect DG0 function space
ff, ind = Function(ff_space), 0 # Initialize source term "ff"

while ind < Cx.shape[1]: # Loop through .csv file for access to well nodes
    dofs = locate_dofs_geometrical(ff_space, lambda x: np.isclose(x.T, [Cx[0,ind],Cy[0,ind],Cz[0,ind]]).all(axis=1)) # locate degree of freedom
    ff.x.array[dofs] = -flow_rate # Source term!
    F += dt*inner(ff,del_p)*dx # Add source term to equation residual
    ind +=1 # Update loop index

#-----------------------------------------------------------------------------------
# Boundaries list: 1=south, 2=east, 3=north, 4=west, 5=base, 6=top, 7=well
#-----------------------------------------------------------------------------------
boundaries = [(1,lambda x: np.isclose(x[1], 0)),
              (2,lambda x: np.isclose(x[0], xl)),
              (3,lambda x: np.isclose(x[1], yl)),
              (4,lambda x: np.isclose(x[0], 0)),
              (5,lambda x: np.isclose(x[2], 0)),
              (6,lambda x: np.isclose(x[2], zl))]

In [ ]:
# DOFs for Boundary Conditions
#-----------------------------------------------------------------------------------
facet_indices, facet_markers = [], []
fdim = mesh.topology.dim - 1
for (marker, locator) in boundaries:
    facets = locate_entities(mesh, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full(len(facets), marker))
facet_indices = np.array(np.hstack(facet_indices), dtype=np.int32)
facet_markers = np.array(np.hstack(facet_markers), dtype=np.int32)
sorted_facets = np.argsort(facet_indices)
facet_tag = meshtags(mesh, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

ds = Measure("ds", domain=mesh, subdomain_data=facet_tag) # External integration measure

In [ ]:
class BoundaryCondition():
    def __init__(self, type, marker, values, ind_1, ind_2):
        self._type = type
        if type == "Dirichlet":
            u_D = values
            facets = np.array(facet_tag.indices[facet_tag.values == marker])
            if ind_1 == 0:
                dofs = locate_dofs_topological(V0.sub(ind_1).sub(ind_2), fdim, facets)
                self._bc = dirichletbc(u_D, dofs, V0.sub(ind_1).sub(ind_2))
            elif ind_1 == 1:
                dofs = locate_dofs_topological(V0.sub(ind_1), fdim, facets)
                self._bc = dirichletbc(u_D, dofs, V0.sub(ind_1))
            else:
                dofs = locate_dofs_topological((V0.sub(ind_1),BDM), fdim, facets)
                self._bc = dirichletbc(u_D, dofs, V0.sub(ind_1))
        elif type == "Neumann":
            if ind_1 == 0:
                self._bc = values*inner(norm, del_u) * ds(marker)
            elif ind_1 == 1:
                self._bc = -values*inner(norm, del_q) * ds(marker)
            else:
                pass
        else:
            raise TypeError("Unknown boundary condition: {0:s}".format(type))
    @property
    def bc(self):
        return self._bc

    @property
    def type(self):
        return self._type

In [ ]:
# Implement all Boundary Conditions
#-----------------------------------------------------------------------------------
BDM, _ = V0.sub(2).collapse() # Collect the BDM function space
no_flux = Function(BDM) # Initinalize flux BC
no_flux.x.array[:] = 0.0 # No flux! homogenous Neumann BC

#-----------------------------------------------------------------------------------
# Boundaries list: 1=south, 2=east, 3=north, 4=west, 5=base, 6=top, 7=well
# Example entry: BoundaryCondition(type, Marker, value, ind_1, ind_2)
# 1. Type is either "Dirichlet" or "Neumann".
# 2. Marker is either 1,2,3,4,5,6 or 7 depending on the boundary.
# 3. ind_1=0 for displacement, ind_1=1 for pressure, or in_1=2 for flux constraint
# 4. ind_2=0 for x-value, ind_2=1 for y_value, or ind_2=2 for z-value.

# Note... Tectonic strain is converted to displacement BC through epsilon*domain_length
#-----------------------------------------------------------------------------------
boundary_conditions = [BoundaryCondition("Dirichlet", 1, ScalarType(0), 0, 1),
                           BoundaryCondition("Dirichlet", 2, ScalarType(-eps_h*xl), 0, 0),
                           BoundaryCondition("Dirichlet", 3, ScalarType(-eps_H*yl), 0, 1),
                           BoundaryCondition("Dirichlet", 4, ScalarType(0), 0, 0),
                           BoundaryCondition("Dirichlet", 5, ScalarType(0), 0, 2),
                           BoundaryCondition("Neumann", 6, ScalarType(Sv), 0, 2),
                           BoundaryCondition("Neumann", 1, ScalarType(Pp), 1, 0),
                           BoundaryCondition("Neumann", 2, ScalarType(Pp), 1, 0),
                           BoundaryCondition("Neumann", 3, ScalarType(Pp), 1, 0),
                           BoundaryCondition("Dirichlet", 4, no_flux, 2, 0),
                           BoundaryCondition("Dirichlet", 5, no_flux, 2, 2),
                           BoundaryCondition("Dirichlet", 6, no_flux, 2, 2)]

bcs = []

# Loop through all bounday conditions and apply them to equations
#-----------------------------------------------------------------------------------
for condition in boundary_conditions:
    if condition.type == "Dirichlet":
        bcs.append(condition.bc)
    else:
        F += condition.bc # If Neumman --> Add to equation residual

In [ ]:
# Transient solution
#-----------------------------------------------------------------------------------
sol_time = np.array([dt, 5*dt, 10*dt, 15*dt, 20*dt, 25*dt, 30*dt, 35*dt, 40*dt, 45*dt, tmax])
file = io.XDMFFile(mesh.comm, "Solution/Depletion_solution2.xdmf", "w") # Output file --> visualize in Paraview
file.write_mesh(mesh) # Attach mesh to the file

uk1_sol = wk1.sub(0) # Current displacement solution vector --> t = k+1
pk1_sol = wk1.sub(1) # Current pressure solution vector --> t = k+1
qk1_sol = wk1.sub(2) # Current flux solution vector --> t = k+1

uk1_sol.name = "Displacement"
pk1_sol.name = "Pore Pressure"
qk1_sol.name = "Darcy Flux"

wk.x.array[:] = wk1.x.array # Initialize previous solution

problem = NonlinearProblem(F, wk1, bcs) # Newton iteration in Fenicsx requires nonlinear problem
solver = NewtonSolver(mesh.comm, problem) # Newton method
solver.convergence_criterion = "incremental" # Use either "incremental" or "residual"
solver.rtol = 1e-4 # Relative tolerance
solver.atol = 1e-6 # Absolute tolerance
solver.report = True # Output solver report
solver.max_it = 15 # Stop if no solution within 15 iterations

ksp = solver.krylov_solver
opts = Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

print("Beginning Simulation..." + "\n")
import time
t_i= time.time()

while (t<tmax):
    t += dt # Update time
    r = solver.solve(wk1) # Solve the problem!
    
    print(f"Step {int(t/dt)}: num iterations: {r[0]}: time: {t/86400} day")
    
    u_sol,p_sol,q_sol = split(wk1) # Nodal solution at t = k+1
    
    # Output all variables --> visualize in Paraview
    expr_1 = Expression(sigma(u_sol,p_sol), S.element.interpolation_points)
    stress = Function(S)
    stress.interpolate(expr_1)
    stress.name = "Total Stress"
    
    expr_2 = Expression(sigma_eff(u_sol,p_sol), S.element.interpolation_points)
    stress_eff = Function(S)
    stress_eff.interpolate(expr_2)
    stress_eff.name = "Effective Stress"
    
    expr_3 = Expression(epsilon(u_sol), S.element.interpolation_points)
    strain = Function(S)
    strain.interpolate(expr_3)
    strain.name = "Strain"
    
    expr_4 = Expression(epsilon_v(u_sol), C.element.interpolation_points)
    strain_v = Function(C)
    strain_v.interpolate(expr_4)
    strain_v.name = "Volumetric Strain"
    
    expr_5 = Expression(porosity(u_sol,p_sol), C.element.interpolation_points)
    poro = Function(C)
    poro.interpolate(expr_4)
    poro.name = "Delta Porosity"
    
    t_day = t/86400 # Convert seconds to days
    # Write all of the above to output file
    if np.isin(t,sol_time) == True:
        file.write_function(uk1_sol,t_day)
        file.write_function(pk1_sol,t_day)
        file.write_function(qk1_sol,t_day)
        file.write_function(stress,t_day)
        file.write_function(stress_eff,t_day)
        file.write_function(strain,t_day)
        file.write_function(strain_v,t_day)
        file.write_function(poro,t_day)
    
    wk.x.array[:] = wk1.x.array # Update previous solution with current solution
file.close()

t_sim = time.time() - t_i
print("\n" + "Done: Solution Converged!")
print("Elaspsed Time [min]: ", t_sim/60)